In [ ]:
import torch
import torchvision
import PIL
import os
from matplotlib import pyplot as plt
import cv2
import random
import warnings
import timm
from torch import nn
import numpy as np
from tqdm import tqdm_notebook
from torchvision.models import mobilenet_v3_small
from torchsummary import summary


warnings.filterwarnings('ignore')

device='cuda' if torch.cuda.is_available() else 'cpu'

print(f'Using: {device}')


In [ ]:
class dataset():
    def __init__(self,src):
        self.src=src
        self.images=os.listdir(src)
        #self.transforms=torchvision.models.MobileNet_V3_Small_Weights.IMAGENET1K_V1.transforms()
        self.transforms=torchvision.transforms.Compose([torchvision.transforms.Resize((256,256)),
                                                       torchvision.transforms.CenterCrop((224,224))])
    
    def __len__(self):
        return len(self.images)

    def apply_mosaic(self,image, top_left_x, top_left_y, width, height, mosaic_size):
        """
        Apply a mosaic effect to a specific area of the image.
    
        :param image: The original image.
        :param top_left_x: Top left x-coordinate of the mosaic area.
        :param top_left_y: Top left y-coordinate of the mosaic area.
        :param width: Width of the mosaic area.
        :param height: Height of the mosaic area.
        :param mosaic_size: Size of the mosaic blocks.
        """
        # Extract the region of interest
        roi = image[top_left_y:top_left_y+height, top_left_x:top_left_x+width]
    
        #plt.imshow(roi)
        
        # Resize to mosaic size (downscale)
        roi_small = cv2.resize(roi, (mosaic_size, mosaic_size), interpolation=cv2.INTER_LINEAR)
    
        # Resize back to original size (upscale)
        mosaic = cv2.resize(roi_small, (width, height), interpolation=cv2.INTER_NEAREST)
        
        # Place the mosaic back into the original image
        image[top_left_y:top_left_y+height, top_left_x:top_left_x+width] = mosaic
        return image


    def make_ds(self):
        for idx in range(len(self.images)):
        
            img=cv2.imread(self.src+"/"+self.images[idx])
          
        
            downsize=random.randint(0,1)

            if(downsize):
                image=img
                
                original_height, original_width = image.shape[:2]

                # Choose the scale factor (5 or 6)
                scale_factor = 6
                
                # Calculate the new dimensions
                new_width = original_width // scale_factor
                new_height = original_height // scale_factor
                
                # Resize the image down by the scale factor
                downscaled_image = cv2.resize(image, (new_width, new_height), interpolation=cv2.INTER_LINEAR)
                
                # Resize the image back to the original size
                upscaled_image = (cv2.resize(downscaled_image, (original_width, original_height), interpolation=cv2.INTER_LINEAR))

                upscaled_image=torch.from_numpy(cv2.Canny(upscaled_image,t_lower,t_upper)[np.newaxis,...])

                upscaled_image=torch.vstack((upscaled_image,upscaled_image,upscaled_image))
                return self.transforms(upscaled_image.type(torch.float32)), torch.tensor([0,0,1]).type(torch.float32)
                                
            else:
                img_height, img_width = img.shape[:2]
        
                # Define the size of the mosaic area
                mosaic_width = random.randint(50,200)
                mosaic_height = random.randint(50,200)
                mosaic_size = random.randint(5,10)  # Size of each mosaic block
                
                # Randomly pick a spot
                top_left_x = random.randint(0, img_width - mosaic_width)
                top_left_y = random.randint(0, img_height - mosaic_height)
                
                # Apply the mosaic effect
                output_image = self.apply_mosaic(img, top_left_x, top_left_y, mosaic_width, mosaic_height, mosaic_size)

                output_image = torch.from_numpy(cv2.Canny(output_image,t_lower,t_upper)[np.newaxis,...])

                output_image=torch.vstack((output_image,output_image,output_image))
                
                #print(output_image.type(torch.float32).permute(1,2,0).shape)
                #return self.transforms(output_image.type(torch.float32).permute(1,2,0)), pixelize
                return self.transforms(output_image.type(torch.float32)), torch.tensor([0,1,0]).type(torch.float32)

        else:
            #print(torch.from_numpy(img).type(torch.float32).shape)
            #return self.transforms(torch.from_numpy(img).type(torch.float32)), pixelize

            
            img = torch.from_numpy(cv2.Canny(img,t_lower,t_upper)[np.newaxis,...])
            img = torch.vstack((img,img,img))
            
            return self.transforms(img.type(torch.float32)),torch.tensor([1,0,0]).type(torch.float32)
        